# **Projet 8 - Déployez un modèle dans le cloud : Notebook cloud**

**Table des matières**<a id='toc0_'></a>    
- 1. [Introduction](#toc1_)    
  - 1.1. [Contexte](#toc1_1_)    
  - 1.2. [Mission](#toc1_2_)    
  - 1.3. [Contraintes](#toc1_3_)    
  - 1.4. [NOTE](#toc1_4_)    
- 2. [Démarrage de la session Spark et importation des librairies](#toc2_)    
  - 2.1. [Démarrage de la session Spark](#toc2_1_)    
  - 2.2. [Importation des librairies](#toc2_2_)    
- 3. [Définition des PATH pour le chargement des images et l'enregistrement des résultats](#toc3_)    
- 4. [Traitement des données](#toc4_)    
  - 4.1. [Chargement des données](#toc4_1_)    
  - 4.2. [Préparation du modèle](#toc4_2_)    
    - 4.2.1. [Fonction pour la création du modèle](#toc4_2_1_)    
  - 4.3. [Définition du processus de chargement des images et application de leur featurisation à travers l'utilisation de pandas UDF](#toc4_3_)    
- 5. [Exécution des actions d'extraction de features](#toc5_)    
  - 5.1. [Extraction sur les 50 images](#toc5_1_)    
  - 5.2. [Réduction dimensionnelle](#toc5_2_)    
  - 5.3. [Enregistrement des données](#toc5_3_)    
- 6. [Validation des résultats](#toc6_)    
  - 6.1. [Chargement des données](#toc6_1_)    
  - 6.2. [Création d'un colonne par composante](#toc6_2_)    
  - 6.3. [Sauvegarde des résultats](#toc6_3_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=2
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

## 1. <a id='toc1_'></a>[Introduction](#toc0_)
### 1.1. <a id='toc1_1_'></a>[Contexte](#toc0_)
Vous êtes Data Scientist dans une très jeune start-up de l'AgriTech, nommée  "Fruits!", qui cherche à proposer des solutions innovantes pour la récolte des fruits.

La volonté de l’entreprise est de préserver la biodiversité des fruits en permettant des traitements spécifiques pour chaque espèce de fruits en développant des robots cueilleurs intelligents.  

Votre start-up souhaite dans un premier temps se faire connaître en mettant à disposition du grand public une application mobile qui permettrait aux utilisateurs de prendre en photo un fruit et d'obtenir des informations sur ce fruit.

Pour la start-up, cette application permettrait de sensibiliser le grand public à la biodiversité des fruits et de mettre en place une première version du moteur de classification des images de fruits.  

De plus, le développement de l’application mobile permettra de construire une première version de l'architecture Big Data nécessaire.

### 1.2. <a id='toc1_2_'></a>[Mission](#toc0_)
Vous êtes donc chargé de vous approprier les travaux réalisés par l’alternant et de compléter la chaîne de traitement.

Il n’est pas nécessaire d’entraîner un modèle pour le moment.

L’important est de mettre en place les premières briques de traitement qui serviront lorsqu’il faudra passer à l’échelle en termes de volume de données !

### 1.3. <a id='toc1_3_'></a>[Contraintes](#toc0_)

Lors de son brief initial, Paul vous a averti des points suivants :
- Vous devrez tenir compte dans vos développements du fait que le volume de données va augmenter très rapidement après la livraison de ce projet. Vous continuerez donc à développer des scripts en Pyspark et à utiliser le cloud AWS pour profiter d’une architecture Big Data (EMR, S3, IAM). Si vous préférez, vous pourrez transférer les traitements dans un environnement Databricks

- Vous devez faire une démonstration de la mise en place d’une instance EMR opérationnelle, ainsi qu’ expliquer pas à pas le script PySpark, que vous aurez complété : 
    - d’un traitement de diffusion des poids du modèle Tensorflow sur les clusters (broadcast des “weights” du modèle) qui avait été oublié par l’alternant. Vous pourrez vous appuyer sur l’article “Distributed model inference using TensorFlow Keras” disponible dans les ressources
    - d’une étape de réduction de dimension de type PCA en PySpark 
    
- Vous respecterez les contraintes du RGPD : dans notre contexte, vous veillerez à paramétrer votre installation afin d’utiliser des serveurs situés sur le territoire européen 

- Votre retour critique de cette solution sera également précieuse, avant de décider de la généraliser

- La mise en œuvre d’une architecture Big Data de type EMR engendrera des coûts. Vous veillerez donc à ne maintenir l’instance EMR opérationnelle que pour les tests et les démos.

### 1.4. <a id='toc1_4_'></a>[NOTE](#toc0_)
<u>**Afin de limiter les coûts, le jeu de données à été restreint. Ici il ne sera utilisé que 5 photos de 10 fruits différents**  
Soit 50 photos au total</u>  

## 2. <a id='toc2_'></a>[Démarrage de la session Spark et importation des librairies](#toc0_)
### 2.1. <a id='toc2_1_'></a>[Démarrage de la session Spark](#toc0_)

In [1]:
# L'exécution de cette cellule démarre l'application Spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1706104113929_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
%%info

ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1706104113929_0001,pyspark,idle,Link,Link,None,✔


In [3]:
%%bash
uname -a

Linux jupyterhub 4.14.334-252.552.amzn2.x86_64 #1 SMP Tue Jan 2 17:47:37 UTC 2024 x86_64 x86_64 x86_64 GNU/Linux


### 2.2. <a id='toc2_2_'></a>[Importation des librairies](#toc0_)

In [4]:
import pandas as pd
import numpy as np
import io
import os
import tensorflow as tf
from PIL import Image
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras import Model
from pyspark.sql.functions import col, pandas_udf, PandasUDFType, element_at, split, udf

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.types import ArrayType, FloatType
from pyspark.ml.feature import PCA

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
from typing import Iterator

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
%%bash
pip install tensorflow==2.7.4
#pip install tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 496.1/496.1 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 104.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 97.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 463.1/463.1 kB 53.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 82.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 89.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 103.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [9]:
%%bash
pip show tensorflow keras

Name: tensorflow
Version: 2.7.4
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /opt/conda/lib/python3.9/site-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, keras, keras-preprocessing, libclang, numpy, opt-einsum, protobuf, six, tensorboard, tensorflow-estimator, tensorflow-io-gcs-filesystem, termcolor, typing-extensions, wheel, wrapt
Required-by: 
---
Name: keras
Version: 2.7.0
Summary: Deep learning for humans.
Home-page: https://keras.io/
Author: Keras team
Author-email: keras-users@googlegroups.com
License: Apache 2.0
Location: /opt/conda/lib/python3.9/site-packages
Requires: 
Required-by: tensorflow


In [10]:
%%bash
pip show Pandas pillow  pyarrow

Name: pandas
Version: 1.4.2
Summary: Powerful data structures for data analysis, time series, and statistics
Home-page: https://pandas.pydata.org
Author: The Pandas Development Team
Author-email: pandas-dev@python.org
License: BSD-3-Clause
Location: /opt/conda/lib/python3.9/site-packages
Requires: numpy, python-dateutil, pytz
Required-by: autovizwidget, hdijupyterutils, sparkmagic
---
Name: Pillow
Version: 9.1.1
Summary: Python Imaging Library (Fork)
Home-page: https://python-pillow.org
Author: Alex Clark (PIL Fork Author)
Author-email: aclark@python-pillow.org
License: HPND
Location: /opt/conda/lib/python3.9/site-packages
Requires: 
Required-by: matplotlib


In [11]:
%%bash
pip install pyarrow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 49.8 MB/s eta 0:00:00


## 3. <a id='toc3_'></a>[Définition des PATH pour le chargement des images et l'enregistrement des résultats](#toc0_)

Pour accéder aux données sur s3, nous utilisions les liens comme ci elles étaient présente en localement

In [12]:
PATH = 's3://oc-p8-test-bucket/'
PATH_Data   = PATH+'Test/'
SPATH_Result = PATH+'Results/'


print('PATH:        '+\
      PATH+'\nPATH_Data:   '+\
      PATH_Data+'\nPATH_Result: '+PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

PATH:        s3://oc-p8-test-bucket/
PATH_Data:   s3://oc-p8-test-bucket/Test/
PATH_Result: s3://oc-p8-test-bucket/Results/

## 4. <a id='toc4_'></a>[Traitement des données](#toc0_)
### 4.1. <a id='toc4_1_'></a>[Chargement des données](#toc0_)

In [13]:
images = spark.read.format("binaryFile") \
  .option("pathGlobFilter", "*.jpg") \
  .option("recursiveFileLookup", "true") \
  .load(PATH_Data)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
images.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------------+------+--------------------+
|                path|   modificationTime|length|             content|
+--------------------+-------------------+------+--------------------+
|s3://oc-p8-test-b...|2024-01-08 11:24:43|  7353|[FF D8 FF E0 00 1...|
|s3://oc-p8-test-b...|2024-01-08 11:25:15|  7350|[FF D8 FF E0 00 1...|
|s3://oc-p8-test-b...|2024-01-08 14:27:20|  7349|[FF D8 FF E0 00 1...|
|s3://oc-p8-test-b...|2024-01-08 14:28:20|  7348|[FF D8 FF E0 00 1...|
|s3://oc-p8-test-b...|2024-01-08 14:27:45|  7328|[FF D8 FF E0 00 1...|
+--------------------+-------------------+------+--------------------+
only showing top 5 rows

In [15]:
# Création d'une colonne label et sélection uniquement de path et label : 

# Ajout d'une nouvelle colonne 'label' au dataframe images
images = images.withColumn('label', element_at(split(images['path'], '/'),-2))

# Impression des résultats
images.select('path','label').show(5,False)

# Impression du schéma du dataframe
print(images.printSchema())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------------------------------------------+----------+
|path                                                |label     |
+----------------------------------------------------+----------+
|s3://oc-p8-test-bucket/Test/Watermelon/r_106_100.jpg|Watermelon|
|s3://oc-p8-test-bucket/Test/Watermelon/r_109_100.jpg|Watermelon|
|s3://oc-p8-test-bucket/Test/Watermelon/r_108_100.jpg|Watermelon|
|s3://oc-p8-test-bucket/Test/Watermelon/r_107_100.jpg|Watermelon|
|s3://oc-p8-test-bucket/Test/Watermelon/r_95_100.jpg |Watermelon|
+----------------------------------------------------+----------+
only showing top 5 rows

root
 |-- path: string (nullable = true)
 |-- modificationTime: timestamp (nullable = true)
 |-- length: long (nullable = true)
 |-- content: binary (nullable = true)
 |-- label: string (nullable = true)

None

In [16]:
# Vérification du nombre d'images (50 attendu) : 
images.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

15090

### 4.2. <a id='toc4_2_'></a>[Préparation du modèle](#toc0_)

Nous créons également ici, une diffusion des poids du modèles (brodcasting) à travers les différents neouds de calcul.  
Cela nous permets l'accélération de l'entrainement du modèle sur de grands ensembles de données.

In [17]:
from tensorflow.keras.applications import MobileNetV2

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
# Création du modèle MobileNetV2 avec l'ensemble des couches : 
base_model  = MobileNetV2(weights='imagenet',
                    include_top=True,
                    input_shape=(224, 224, 3))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

14536120/14536120 [==============================] - 1s 0us/step

In [19]:
# Création du modèle spécifique (retrait de la denrière couche): 
model = Model(inputs=base_model.input,
              outputs=base_model.layers[-2].output)

# Diffusion des poids du modèle :
brodcast_weights = sc.broadcast(model.get_weights())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
# Résumé du modèle
model.summary()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                             

#### 4.2.1. <a id='toc4_2_1_'></a>[Fonction pour la création du modèle](#toc0_)

In [21]:
def model_fn():
    """
    Charge et configure un modèle pré-entraîné MobileNetV2 pour l'extraction de caractéristiques.

    Returns:
        Model: Un modèle Keras configuré pour l'extraction de caractéristiques.
    """
    
    # Chargement du modèle MobileNetV2 pré-entraîné sur ImageNet :
    model = MobileNetV2(weights='imagenet',
                        include_top=True,
                        input_shape=(224, 224, 3))
    
    # Désactivation de l'entraînement des couches existantes du modèle : 
    for layer in model.layers:
        layer.trainable = False
        
    # Création d'un nouveau modèle basé sur les caractéristiques du modèle existant : 
    new_model = Model(inputs=model.input,
                      outputs=model.layers[-2].output)
    
    # Initialisation des poids du nouveau modèle avec les poids diffusés (broadcasted) : 
    new_model.set_weights(brodcast_weights.value)
    
    return new_model

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### 4.3. <a id='toc4_3_'></a>[Définition du processus de chargement des images et application de leur featurisation à travers l'utilisation de pandas UDF](#toc0_)

L'empilement des appels est la suivante :
- Pandas UDF
  - Featuriser une série d'images pd.Series
    - Prétraitement d'une image

In [22]:
# Fonction de préparation des images : 
def preprocess(content):
    """
    Prétraite le contenu brut d'une image pour la prédiction.

    Args:
        content (bytes): Contenu brut de l'image au format bytes.

    Returns:
        numpy.ndarray: Tableau NumPy représentant l'image prétraitée.
    """
    
    # Ouverture de l'image à partir du contenu brut et la redimensionne : 
    img = Image.open(io.BytesIO(content)).resize([224, 224])
    
    # Convertion de l'image en tableau NumPy : 
    arr = img_to_array(img)
    
    # Application du prétraitement spécifique au modèle : 
    return preprocess_input(arr)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
# Fonction de création de features : 
def featurize_series(model, content_series):
    """
    Génère des caractéristiques (features) à partir d'une série d'images brutes en utilisant le modèle spécifié.

    Args:
        model (keras.Model): Modèle Keras utilisé pour générer les caractéristiques.
        content_series (pandas.Series): Série Pandas contenant du contenu brut d'images.

    Returns:
        pandas.Series: Série Pandas contenant les caractéristiques générées à partir des images.
    """
    # Prétraitement de chaque image dans la série : 
    input = np.stack(content_series.map(preprocess))
    
    # Préditction des caractéristiques : 
    preds = model.predict(input)
    
    # Pour certaines couches, les caractéristiques de sortie seront des tenseurs multidimensionnels.
    # Nous aplatissions les tenseurs de caractéristiques en vecteurs pour un stockage plus facile dans les DataFrames Spark.
    output = [p.flatten() for p in preds]
    
    # Renvoir des caractéristiques sous forme de série Pandas : 
    return pd.Series(output)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
@pandas_udf('array<float>', PandasUDFType.SCALAR_ITER)
def featurize_udf(content_series_iter: Iterator[pd.Series]) -> Iterator[pd.Series]:
    """
    Génère des caractéristiques à partir d'un itérateur sur des lots de données d'images en utilisant un modèle pré-chargé.

    Args:
        content_series_iter (Iterator[pandas.Series]): Un itérateur sur des lots de données, où chaque lot
                                                      est une série Pandas de données d'image.

    Yields:
        pandas.Series: Une série Pandas contenant les caractéristiques générées pour chaque lot d'images.
    """
    
    # Avec les Pandas UDF de type Scalar Iterator, nous pouvons charger le modèle une fois et le réutiliser
    # pour plusieurs lots de données. Cela amortit les frais généraux de chargement de gros modèles.
   
    model = model_fn()
    for content_series in content_series_iter:
        yield featurize_series(model, content_series)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

/mnt1/yarn/usercache/livy/appcache/application_1706104113929_0001/container_1706104113929_0001_01_000001/pyspark.zip/pyspark/sql/pandas/functions.py:392: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.

## 5. <a id='toc5_'></a>[Exécution des actions d'extraction de features](#toc0_)

Comme précédemment précisé, l'extraction des caractéristiques des images sera réalisée sur 50 images: 10 fruits avec 5 images par fruit.  
Ces images étant issues de jeu de données test.

### 5.1. <a id='toc5_1_'></a>[Extraction sur les 50 images](#toc0_)


In [25]:
# Extraction des features en utilisant 20 exécuteurs : 
features_df = images.repartition(20).select(
    col("path"),
    col("label"),
    featurize_udf("content").alias("features")
)

# Visualisation des 5 premières lignes du DataFrame obtenu : 
features_df.show(5, truncate=True)

# Vérification du nombre d'images (50 attendues) : 
print(f"Nombre d'images : {features_df.count()}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----------+--------------------+
|                path|     label|            features|
+--------------------+----------+--------------------+
|s3://oc-p8-test-b...|Watermelon|[1.0325117, 0.390...|
|s3://oc-p8-test-b...|Watermelon|[0.015379974, 0.3...|
|s3://oc-p8-test-b...|Watermelon|[0.04004709, 0.15...|
|s3://oc-p8-test-b...|Watermelon|[0.02610845, 0.41...|
|s3://oc-p8-test-b...| Raspberry|[0.083246835, 0.6...|
+--------------------+----------+--------------------+
only showing top 5 rows

Nombre d'images : 15090

### 5.2. <a id='toc5_2_'></a>[Réduction dimensionnelle](#toc0_)
A l'aide d'une PCA avec 138 composantes. 

Les 138 composantes ayant étauent définies lors du test local, permettant d'atteindre 95% de la variance expliquée. 

In [26]:
# Création d'une fonction de conversion de la colonne 'features' en vecteur : 
features_to_vector_udf = udf(lambda arr: Vectors.dense(arr), VectorUDT())

# Application de la fonction au DataFrame et création d'une nouvelle colonne : 
features_df = features_df.withColumn("features_vector", features_to_vector_udf("features"))

# Création d'un modèle PCA avec les 138 composantes principales pour atteindre 95% de la variance : 
pca = PCA(k=138, inputCol="features_vector", outputCol="vectorized_components_pca_features")

# Application de la PCA sur le DataFrame : 
pca = pca.fit(features_df)
features_df = pca.transform(features_df)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
# Affichage des 5 premières lignes : 
features_df.show(5, truncate=True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----------+--------------------+--------------------+----------------------------------+
|                path|     label|            features|     features_vector|vectorized_components_pca_features|
+--------------------+----------+--------------------+--------------------+----------------------------------+
|s3://oc-p8-test-b...|Watermelon|[1.0325117, 0.390...|[1.03251171112060...|              [-2.2313900934634...|
|s3://oc-p8-test-b...|Watermelon|[0.015379974, 0.3...|[0.01537997368723...|              [0.91725650609218...|
|s3://oc-p8-test-b...|Watermelon|[0.04004709, 0.15...|[0.04004709050059...|              [-2.4707839026256...|
|s3://oc-p8-test-b...|Watermelon|[0.02610845, 0.41...|[0.02610844932496...|              [-1.4624585393730...|
|s3://oc-p8-test-b...| Raspberry|[0.32444865, 0.26...|[0.32444864511489...|              [-0.3179525611560...|
+--------------------+----------+--------------------+--------------------+----------------------------------+
o

In [28]:
# Restructuration des vecteurs composantes PCA en array : 

# Fonction de conversion vector to array : 
vector_to_array_udf = udf(lambda vec: vec.toArray().tolist(), ArrayType(FloatType()))

# Application de la fonction pour créer une nouvelle colonne pca_features : 
features_df = features_df.withColumn("pca_features", vector_to_array_udf("vectorized_components_pca_features"))

# Création du DataFrame final : 
final_df = features_df.select("path", "label", "pca_features")
final_df.show(5)
final_df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----------+--------------------+
|                path|      label|        pca_features|
+--------------------+-----------+--------------------+
|s3://oc-p8-test-b...| Watermelon|[-2.23139, 7.9415...|
|s3://oc-p8-test-b...| Watermelon|[0.91725653, 6.94...|
|s3://oc-p8-test-b...| Watermelon|[-2.470784, 6.979...|
|s3://oc-p8-test-b...| Watermelon|[-1.4624585, 4.82...|
|s3://oc-p8-test-b...|Cauliflower|[-3.2852638, 2.08...|
+--------------------+-----------+--------------------+
only showing top 5 rows

root
 |-- path: string (nullable = true)
 |-- label: string (nullable = true)
 |-- pca_features: array (nullable = true)
 |    |-- element: float (containsNull = true)

### 5.3. <a id='toc5_3_'></a>[Enregistrement des données](#toc0_)
Nous enregistrerons les données au format parquet.

In [29]:
# Enregistrement des données : 
final_df.write.mode("overwrite").parquet(PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 6. <a id='toc6_'></a>[Validation des résultats](#toc0_)
### 6.1. <a id='toc6_1_'></a>[Chargement des données](#toc0_)

In [30]:
# Chargement des données depuis path_result : 
#df = pd.read_parquet(PATH_Result, engine='fastparquet')
#df = pd.read_parquet(PATH_Result)

df = pd.read_parquet(PATH_Result, engine='pyarrow')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
print(f'Dimension de df : {df.shape}')
df.head()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Dimension de df : (15090, 3)
                                                path  ...                                       pca_features
0  s3://oc-p8-test-bucket/Test/Watermelon/r_53_10...  ...  [-2.23139, 7.941544, -6.305119, 1.1899054, -2....
1  s3://oc-p8-test-bucket/Test/Watermelon/r_176_1...  ...  [0.91725653, 6.9456825, -7.7219815, -0.7499954...
2  s3://oc-p8-test-bucket/Test/Watermelon/130_100...  ...  [-2.470784, 6.9792175, -7.4942293, 1.9953984, ...
3  s3://oc-p8-test-bucket/Test/Watermelon/249_100...  ...  [-1.4624585, 4.8205585, -5.260026, 1.6868976, ...
4   s3://oc-p8-test-bucket/Test/Raspberry/89_100.jpg  ...  [0.71290225, 4.16823, 1.0440894, 10.545405, -0...

[5 rows x 3 columns]

In [32]:
# Validation de la dimension des pca_features : 
print(f"Dimension des pca_features : {df.loc[0, 'pca_features'].shape}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Dimension des pca_features : (138,)

### 6.2. <a id='toc6_2_'></a>[Création d'un colonne par composante](#toc0_)

In [33]:
columns = []
nbr_composantes = 138

# Récupération des series de composates : 
for i in range(nbr_composantes): 
    columns.append(pd.Series(df['pca_features'].apply(lambda x: x[i]), name=f'pca_feature_{i+1}'))

# Concaténation des colonnes au DataFrame df : 
df = pd.concat([df] + columns, axis=1)

# Suppression de pca_features : 
df = df.drop('pca_features', axis=1)

# Affichage : 
print(f"Dimension de df : {df.shape}")
df.head()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Dimension de df : (15090, 140)
                                                path  ... pca_feature_138
0  s3://oc-p8-test-bucket/Test/Watermelon/r_53_10...  ...       -0.377717
1  s3://oc-p8-test-bucket/Test/Watermelon/r_176_1...  ...       -0.107848
2  s3://oc-p8-test-bucket/Test/Watermelon/130_100...  ...        0.579057
3  s3://oc-p8-test-bucket/Test/Watermelon/249_100...  ...       -0.188433
4   s3://oc-p8-test-bucket/Test/Raspberry/89_100.jpg  ...        0.212016

[5 rows x 140 columns]

In [34]:
# Répartition par labels : 
df['label'].value_counts()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Plum 3                304
Tomato 3              246
Melon Piel de Sapo    246
Cherry Rainier        246
Strawberry Wedge      246
                     ... 
Mango Red             142
Plum 2                142
Apple Braeburn        130
Ginger Root            99
Passion Fruit          61
Name: label, Length: 89, dtype: int64

### 6.3. <a id='toc6_3_'></a>[Sauvegarde des résultats](#toc0_)
Sauvegarde du DataFrame au format CSV dans le bucket s3

In [35]:
# Enregistrement du DataFrame en tant que fichier CSV sur S3
df.to_csv(PATH_Result + '/df_results_cloud.csv', index=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
# Vérification de l'enregistrement : 
df = pd.read_csv(PATH_Result + '/df_results_cloud.csv')

# Affichage des 5 premières lignes : 
df.head()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                                                path  ... pca_feature_138
0  s3://oc-p8-test-bucket/Test/Watermelon/r_53_10...  ...       -0.377717
1  s3://oc-p8-test-bucket/Test/Watermelon/r_176_1...  ...       -0.107848
2  s3://oc-p8-test-bucket/Test/Watermelon/130_100...  ...        0.579057
3  s3://oc-p8-test-bucket/Test/Watermelon/249_100...  ...       -0.188433
4   s3://oc-p8-test-bucket/Test/Raspberry/89_100.jpg  ...        0.212016

[5 rows x 140 columns]

===================================================================

In [37]:
%%bash
pip list

Package                       Version
----------------------------- -----------
absl-py                       2.1.0
aiobotocore                   1.3.3
aiohttp                       3.8.1
aioitertools                  0.10.0
aiosignal                     1.2.0
alembic                       1.8.0
anyio                         3.6.1
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
asttokens                     2.0.5
astunparse                    1.6.3
async-generator               1.10
async-timeout                 4.0.2
attrs                         21.4.0
autovizwidget                 0.20.0
Babel                         2.10.1
backcall                      0.2.0
backports.functools-lru-cache 1.6.4
beautifulsoup4                4.11.1
bleach                        5.0.0
blinker                       1.4
boto3                         1.21.21
botocore                      1.20.106
brotlipy                      0.7.0
cachetools                    5.2.0
certifi    